# LD Score calculation data analysis

In [707]:
# Testing the outputs of gcta
import pandas as pd
import numpy as np

df = pd.read_csv('gcta_output_1000/chr1.score.ld', sep='\s+', comment='#')
ld = df['ldscore']
print(np.mean(ld))
print(np.std(ld))
print(np.min(ld))
print(np.max(ld))


77.20471790528036
102.53322055587697
-18.7104
930.427


In [595]:
ld = load_ldscores()['ldscore']
print(np.mean(ld))
print(np.std(ld))
print(np.min(ld))
print(np.max(ld))

86.02253763116016
138.0273413724897
-43.3331
3558.8


In [593]:
# Intra-European LD Score Differences
def intra_european_analysis():
    """Calculate the LD scores of the subpopulations and their means
    NOTE: might have to do alignment of SNPs here as well
    """

    pop_dict = {}
    subpop = ['CEU', 'GBR', 'TSI', 'FIN']
    gwas_variants = pd.read_csv('final_ld_variants.txt', sep='\s+', comment='#', header=None)
    for pop in subpop:
        ld_list = []
        for i in range(1,23):
            ld_list += pd.read_csv('gcta_output_1000_{}/chr{}.score.ld'.format(pop, str(i)), sep='\s+', comment='#')['ldscore'].tolist()
        pop_dict[pop] = ld_list
    
    for pop in pop_dict.keys():
        print(pop, np.mean(pop_dict[pop]))
    
    correlation_matrix = np.corrcoef([pop_dict[elt] for elt in subpop])
    print(correlation_matrix)

intra_european_analysis()

CEU 108.13776851176041
GBR 106.45838399603205
TSI 100.85018480310791
FIN 125.16354713420286


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4,) + inhomogeneous part.

In [592]:
pop_dict = {}
subpop = ['CEU', 'GBR', 'TSI', 'FIN']
gwas_variants = pd.read_csv('final_ld_variants.txt', sep='\s+', comment='#', header=None)
for pop in subpop:
    ld_list = []
    for i in range(1,2):
        ld_list += pd.read_csv('gcta_output_1000_{}/chr{}.score.ld'.format(pop, str(i)), sep='\s+', comment='#')['ldscore'].tolist()
    pop_dict[pop] = ld_list

for pop in pop_dict.keys():
    print(pop, np.mean(pop_dict[pop]))

CEU 97.2441202535181
GBR 94.7820603744863
TSI 90.16067657652862
FIN 113.47286522328261


# ld score regression main code

Thoughts: 
1. This is a regression, with inputs being the scaled ld scores + number of individuals, and output being the chi-squared values 
2. LD scores will all be taken from gcta_outputs_1000, since that appears to be what the paper has been doing and the GWAS summary statistics
    that are being provided don't have ld scores
3. Calculating the standard error will require some tricks with jackknifing, which is more complicated than just grabbing the standard error from sm.OLS's output


In [208]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [211]:
# Regression
def regression_gwas(chi2, ldscores, M, N, weights):
    """Calculates ldscore regression based on the formula given
    Parameters
    chi2: the chi2 outputs
    ldscores: ld scores from gcta, calculated from 1000 Genomes EUR reference data
    M: Number of SNPs
    N: Number of individuals
    weights: weights for regression that deal with correlation and heteroskedasticity

    Returns:
    h2: heritability, slope of the regression
    a: bias term
    """

    

    X = sm.add_constant(ldscores)
    model = sm.WLS(chi2, X, weights=weights)
    results = model.fit()

    return results

In [737]:
# Load in ld scores
def load_ldscores():
    """Loads in all ld scores from all chromosomes
    
    returns: dataframe of ldscores"""

    ldscores = pd.read_csv('gcta_output/chr1.score.ld', sep='\s+', comment='#')
    for i in range(2, 23):
        ldscores = pd.concat([ldscores, pd.read_csv('gcta_output_1000/chr{}.score.ld'.format(str(i)), sep='\s+', comment='#')])
    
    return ldscores

In [485]:
# Calculate Chi2
from scipy.stats import chi2

def calc_chi2(gwas, effect_size, standard_error = None, beta=True):
    """Calculates the chi2 row for a GWAS summary statistic
    Parameters:
    gwas: the gwas summary statistics as a dataframe. 
    effect_size: the name of the effect size column
    standard_error: the name of the standard error column
    beta: whether to use beta and se, or pvalue
    
    Returns:
    gwas: dataframe with the new chi2 values in a 'chi2' column"""

    if beta:
        gwas['chi2'] = np.square(pd.to_numeric(gwas[effect_size]).values / pd.to_numeric(gwas[standard_error]).values).tolist()
    else:
        gwas['chi2'] = chi2.isf(pd.to_numeric(gwas[effect_size]).values, 1).tolist()
    #print([(gwas['BETA'][a], gwas['SE'][a]) for a in [9206,168895,352748,467259,792156,866925,895607,914627,1062022] ])
    return gwas

In [ ]:
# Align files
def align_stats(gwas, ldscores, weights):
    """Returns a cut-down pair of gwas and ldscore dataframes where they only have the shared SNPs
    Parameters:
    gwas: dataframe of gwas statistics
    ldscores: dataframe of ldscore values
    weights: dataframe of the regression weights"""
    common_snps = set(gwas['SNPID']).intersection(set(ldscores['SNP'])).intersection(set(weights))

    # Filter the DataFrames to include only the common SNPs
    gwas = gwas[gwas['SNPID'].isin(common_snps)]
    ldscores = ldscores[ldscores['SNPID'].isin(common_snps)]
    weights = weights[weights.isin(common_snps)]

    return gwas, ldscores, weights

In [ ]:
# Preprocess data
def load_data(args, ldscores):
    """Takes in existing ld scores, loads in new files, then aligns their SNP positions
    Parameters
    args: dictionary with necessary parameters including path of files to load in
    ldscores: ld scores calculated from the 1000 Genomes EUR reference data
    
    Returns:
    gwas: dataframe of gwas statistics
    ldscores: dataframe of ldscore values
    weights: dataframe of the regression weights"""
    # Load in new files
    new_df = calc_chi2(pd.read_csv(args.input_file, sep='\s+', comment='#'))
    weights = pd.read_csv(args.weights_file, sep='\s+', comment='#')

    # Align the two file formats
    return align_stats(new_df, ldscores, weights)

    

In [ ]:
# Plotting
def plot_outputs(pvalues):
    """Plot the outputs of ld score regression in QQ plots"""

In [ ]:
# Process Output
def process_output(results):
    """Takes in results of regression and does data analysis
    Plots QQ plots, prints results
    Parameters:
    results: results of ld score regression
    """

# Misc Utils

In [143]:
# Get HapMap3 reference panel
import time

def load_hapmap():
    """Load in hapmap3 data
    
    Returns:
    List of unique hapmap3 SNPs"""
    # read in all map files
    pops = ['ASW', 'CEU', 'CHB', 'CHD', 'GIH', 'JPT', 'LWK', 'MEX', 'MKK', 'TSI', 'YRI']
    # turn into a list
    df_list = []
    for i in pops:
        df_list.append(pd.read_csv('../hapmap3/hapmap3_r1_b36_fwd.{}.qc.poly.recode.map'.format(i), sep='\s+', comment='#', header=None))
    df = pd.concat(df_list)



    filter_dict = filter_indices(long_read_snps(), accessibility_mask(), df)
    chr_range = [x for x in range(1,23)]
    def keep_row(row):
        """Whether the row satisfies the long read condition
        Parameters:
        row: A row of a dataframe. 0 is the chromosome, 1 is the snpid, 3 is the bp position"""
        if row.name % 10000 == 0:
            print(time.time())
            print(row)
        
        if int(row[0]) not in chr_range:
            return False
        else:
            return filter_dict[int(row[0])][int(row[3]) - 1] # 1 to 0-index
            
            # ranges = long_reads[row[0]]
            # for (start, end) in ranges:
            #     if int(row[3]) >= int(start) * 1e6 and int(row[3]) <= int(end) * 1e6:
            #         return False
            # mask_rows = mask[mask[0] == row[0]]
            # # Check if the SNP position is within the start and end positions of any row
            # in_bed = ((mask_rows[1] <= int(row[3])) & (mask_rows[2] >= int(row[3]))).any()
            
            # # Return False if the SNP is in the BED file (i.e., in_bed is True)
            # return not in_bed

    print("Starting...")
    df = df[df.apply(keep_row, axis=1)]
    # return the unique SNPs
    df_out = pd.DataFrame(list(set(df[1])))
    df_out.to_csv("final_variants.txt", sep=' ', index=False, header=False)

load_hapmap()

Time: 0.0
Chromosome 1
Start Accessibility
Time: 1665575652.6512449
Chromosome 2
Start Accessibility
Time: 1530575659.1940577
Chromosome 3
Start Accessibility
Time: 1624575664.589371
Chromosome 4
Start Accessibility
Time: 1664414359.733654
Chromosome 5
Start Accessibility
Time: 1578075674.5972164
Chromosome 6
Start Accessibility
Time: 1573575679.1957188
Chromosome 7
Start Accessibility
Time: 1658575683.4526908
Chromosome 8
Start Accessibility
Time: 1670575687.3800118
Chromosome 9
Start Accessibility
Time: 1670840878.1652706
Chromosome 10
Start Accessibility
Time: 1676575694.8701987
Chromosome 11
Start Accessibility
Time: 1626075698.5580418
Chromosome 12
Start Accessibility
Time: 1604075702.2120519
Chromosome 13
Start Accessibility
Time: 1653973354.3169866
Chromosome 14
Start Accessibility
Time: 1693779798.1974118
Chromosome 15
Start Accessibility
Time: 1659741670.9518373
Chromosome 16
Start Accessibility
Time: 1679652242.3968937
Chromosome 17
Start Accessibility
Time: 1674213421.621057

In [461]:
# Merge SNPs with LD score SNPs
def merge_with_ld():
    """Takes the filtered HapMap3 SNPs and intersects them with the SNPs used in LD Score calculation

    Saves the outputs as final_ld_variants.txt"""
    
    filtered_variants = pd.read_csv('final_variants.txt', sep='\s+', comment='#', header=None)
    ldscores = load_ldscores()
    intersection = np.intersect1d(filtered_variants[0].values, ldscores['SNP'].values)
    attenuation_df = pd.read_csv('disattenuation/standard_errors.txt', sep='\s+', comment='#')
    pd.DataFrame(intersection).merge(attenuation_df, left_on=0, right_on="SNP").to_csv("final_ld_variants.txt", columns=["SNP"], sep=' ', index=False, header=False)

merge_with_ld()


In [462]:
pd.read_csv('final_ld_variants.txt', sep='\s+', comment='#', header=None)

,0
0,rs1000000
1,rs10000010
2,rs10000012
3,rs1000002
4,rs10000023
...,...
1313459,rs9999978
1313460,rs9999979
1313461,rs9999992
1313462,rs9999995


In [142]:
# Filtered Indices
import numpy as np
import time

def filter_indices(long_reads, mask, hapmap):
    """Takes in the ranges from the long read SNPs and the Accessibility Mask and returns 22 boolean arrays on whether to remove them
    This function is necessary as otherwise the load_hapmap() function will take, conservatively, over a hundred hours
    Parameters:
    long_reads: the long read SNPs
    mask: the accessibility mask
    hapmap: the hapmap data

    Returns:
    filter_dict: dictionary with 22 chromosome filter arrays
    """

    filter_dict = {}
    start = time.time()
    for chr in range(1, 23):
        print("Time:", time.time() - start)
        start = time.time()

        print("Chromosome", chr, flush=True)
        # Get relevant chromosome rows
        mask_rows = mask[mask[0] == chr]
        hapmap_rows = hapmap[hapmap[0] == chr]

        # Get the size of the filter array
        max_end = max(max(mask_rows[2]), max(hapmap_rows[3]))
        filter_dict[chr] = np.array([False] * max_end)
        
        # Apply on Accessibility Mask
        print("Start Accessibility")
        start_indices = mask_rows[1].to_list()
        end_indices = mask_rows[2].to_list()
        for start, end in zip(start_indices, end_indices):
            filter_dict[chr][start:end] = True
        # def row_func(row):
        #     filter_dict[chr][row[1]:row[2]] = [False] * (row[2] - row[1] + 1)
        #     return True
        # mask_rows.apply(lambda row: row_func(row), axis=1) 

        # Apply on Long Reads
        if chr in long_reads.keys():
            for (start, end) in long_reads[chr]:
                start = int(min(start * 1000000, len(filter_dict[chr])))
                end = int(min(end * 1000000, len(filter_dict[chr])))
                filter_dict[chr][start*1000000:end*1000000] = True #[False] * [end - start + 1]
    

    # with open("filtered_dict.npy", "w") as f:
    #     np.save(f, filter_dict)
    return filter_dict

In [118]:
# Remove Long Reads
def long_read_snps():
    """Removes SNPs in the region of long reads from hapmap"""
    long_read = {}
    long_read[1] = [(48, 52)]
    long_read[2] = [(86, 100.5), (134.5, 138), (183, 190)]
    long_read[3] = [(47.5, 50), (83.5, 8), (89, 97.5)]
    long_read[5] = [(44.5, 50.5), (98, 100.5), (129, 132), (135.5, 138.5)]
    long_read[6] = [(57, 64), (140, 142.5)]
    long_read[7] = [(55, 66)]
    long_read[8] = [(8, 12), (43,50)]
    long_read[10] = [(37, 43)]
    long_read[11] = [(87.5, 90.5)]
    long_read[12] = [(33, 40), (109.5, 112)]
    long_read[20] = [(32, 34.5)]

    return long_read

In [97]:
# accessibility mask
def accessibility_mask():
    """Loads in the accessibility mask of the chromosome
    
    Returns: 
    mask: a dictionary of regions where the data should avoid"""
    mask = pd.read_csv('../mask/conversions.bed', sep='\s+', comment='#', header=None)
    mask[0] = mask[0].apply(lambda x : x[3:]) # Convert to GRCh37
    def is_number(value):
        try:
            float(value)
            return True
        except ValueError:
            return False

    # Apply the function to the column of interest
    is_number = mask[0].apply(is_number)

    # Remove rows where 'column_name' is not a number
    mask = mask[is_number]
    mask[0] = pd.to_numeric(mask[0])
    return mask


In [463]:
# Get shared SNPs for regression weights

def regression_weight_1(args):
    """Gets the common SNPs for this GWAS for use in calculating the first regression weight
        Saves the common SNPs in a text file in the folder of the GWAS analysis
    Parameters:
    args: dict with gwas file name
    """

    hapmap_data = pd.read_csv('final_ld_variants.txt', sep='\s+', comment='#', header=None)
    gwas_snps = pd.read_csv(args.gwas_path, sep='\s+', comment='#').dropna(subset=[args.beta, args.se])
    common_snps = pd.DataFrame(np.intersect1d(hapmap_data[0].values, gwas_snps[args.gwas_snp].values))

    common_snps.to_csv(args.gwas_moniker + "/variants.txt", sep=' ', index=False, header=False)



In [240]:
pd.read_csv('Height/variants.txt', sep='\s+', comment='#', header=None)

,0
0,rs1000000
1,rs10000010
2,rs10000012
3,rs10000023
4,rs10000025
...,...
1199172,rs9999963
1199173,rs9999966
1199174,rs9999979
1199175,rs9999992


In [238]:
class args:
    def __init__(self, gwas_path, gwas_moniker, gwas_snp, beta, se):
        self.gwas_path = gwas_path
        self.gwas_moniker = gwas_moniker
        self.gwas_snp = gwas_snp
        self.beta = beta
        self.se = se

In [483]:
# arg = {"gwas_path" : "../GIANT/GIANT_HEIGHT_YENGO_2022_GWAS_SUMMARY_STATS_EUR", "gwas_moniker" : "Height", "gwas_snp" : "RSID"}
arg = args("../gwas_files/pgc.scz.full.2012-04.txt", 'scz', 'snpid', 'pval', 'snpid')
regression_weight_1(arg)

In [447]:
# load in ld scores for the first weight
def get_first_weight(gwas, attenuation_df):
    """Gets the weight from the corresponding folders
    Parameters:
    gwas: gwas path (e.g. Height)"""
    ldscores = pd.read_csv(gwas + '/gcta_output/chr1.score.ld', sep='\s+', comment='#')
    for i in range(2, 23):
        ldscores = pd.concat([ldscores, pd.read_csv(gwas + '/gcta_output/chr{}.score.ld'.format(str(i)), sep='\s+', comment='#')])
    ldscores = ldscores.merge(attenuation_df, left_on="SNP", right_on="SNP")
    return ldscores['ldscore'] #1/ (1 + np.fmax(ldscores))

In [754]:
# Second weight
def second_weight(chi2, ldscores, M, N, first_weight):
    """Calculates ldscore regression based on the formula given
    Parameters
    chi2: the chi2 outputs
    ldscores: ld scores from gcta, calculated from 1000 Genomes EUR reference data
    M: Number of SNPs
    N: Number of individuals
    first_weight: first_weight, the ld score

    Returns:
    h2: heritability, slope of the regression
    a: bias term
    """

    

    X = sm.add_constant(ldscores)
    
    model = sm.OLS(chi2, X)
    results = model.fit()
    print("param:", results.params)
    return np.square(np.multiply(N * min(max((M * results.params[1] / N ), 0), 1) / M, np.fmax(first_weight + 1, 1.0)) + 1)

In [757]:
def get_weight_params(gwas, gwas_data, gwas_file, gwas_snp, effect_size, standard_error):
    """Stopgap function that loads in and merges all regression data for easier testing
    Parameters:
    gwas: gwas path (e.g. Height)
    gwas_data: path to the gwas data folder (e.g. ../GIANT)
    gwas_file: gwas dframe, e.g. GIANT_HEIGHT_YENGO_2022_GWAS_SUMMARY_STATS_EUR
    gwas_snp: label in gwas_file for the RSID
    effect_size: label for the gwas effect size
    standard_error: label for the gwas standard error"""
    start = time.time()

    attenuation_df = pd.read_csv('disattenuation/standard_errors.txt', sep='\s+', comment='#')
    first_ldscores = get_first_weight(gwas, attenuation_df)
    gwas_variants = pd.read_csv(gwas + '/variants.txt', sep='\s+', comment='#', header=None)
    gwas_df = pd.read_csv(gwas_data + '/' + gwas_file, sep='\s+', comment='#')
    
    ldscores = load_ldscores()

    print("Done Loading", time.time() - start)
    start = time.time()

    merged_gwas = calc_chi2(gwas_df.merge(gwas_variants, left_on=gwas_snp, right_on=0).merge(ldscores, left_on=gwas_snp, right_on="SNP").merge(attenuation_df, left_on="SNP", right_on="SNP"), effect_size, standard_error, False)
    
    print("Chi2 Obtained", time.time() - start, "SNPs", len(merged_gwas['SNP']))

    return first_ldscores, gwas_variants, merged_gwas

first_ldscores, gwas_variants, merged_gwas = get_weight_params('bipolar', "../gwas_files", 'pgc.bip.full.2012-04.txt', 'snpid', 'pval', 'snpid')

Done Loading 38.67346262931824
Chi2 Obtained 23.089512586593628 SNPs 1134890


In [764]:
# Scratchpad for testing regressions

start = time.time()

M = gwas_variants[0].size
N = 378
second_weights = second_weight(merged_gwas['chi2'] *  1.187, merged_gwas['ldscore'], M, N, merged_gwas['ldscore'])

print("Second Weight Obtained", time.time() - start)
start = time.time()

weights = 1 / (np.multiply(np.array(np.fmax(first_ldscores + 1, 1.0)), second_weights)) 
#weights = np.array(np.fmax(first_ldscores, 1)) * 2 * second_weights
# print(np.isnan(np.fmax(first_ldscores, 1)).any(), np.isnan(second_weights).any(), np.isnan(weights).any(), np.isnan(merged_gwas['chi2']).any(), np.isnan(merged_gwas['ldscore']).any())

disattenuation = 1 / np.sqrt(np.multiply(np.square(merged_gwas['atten']), second_weights))

weights = np.multiply(weights, disattenuation)

print(weights)
regression_output = regression_gwas(merged_gwas['chi2'] *  1.187, merged_gwas['ldscore'], M, N, weights)

print("Regression completed", time.time() - start)

param: const      1.294905
ldscore    0.000712
dtype: float64
Second Weight Obtained 0.05807900428771973
0          0.069180
1          0.093058
2          0.069805
3          0.046733
4          0.039350
             ...   
1134885    0.015052
1134886    0.040836
1134887    0.031163
1134888    0.038941
1134889    0.056651
Length: 1134890, dtype: float64
Regression completed 0.07553887367248535


C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\139673168.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return np.square(np.multiply(N * min(max((M * results.params[1] / N ), 0), 1) / M, np.fmax(first_weight + 1, 1.0)) + 1)


In [765]:
print(regression_output.params[0], regression_output.params[1]*N)

1.2576125886092844 0.3159255960134252


C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\3766664765.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(regression_output.params[0], regression_output.params[1]*N)


In [660]:
regression_output.params

const      1.046381
ldscore    0.001595
dtype: float64

In [724]:
def get_weights(gwas, gwas_data, gwas_file, gwas_snp, effect_size, standard_error, gc=1, use_beta=True):
    """Get first and second weights
    Parameters:
    gwas: gwas path (e.g. Height)
    gwas_data: path to the gwas data folder (e.g. ../GIANT)
    gwas_file: gwas dframe, e.g. GIANT_HEIGHT_YENGO_2022_GWAS_SUMMARY_STATS_EUR
    gwas_snp: label in gwas_file for the RSID
    effect_size: label for the gwas effect size
    standard_error: label for the gwas standard error
    gc: the genomic control value to reinflate the chi2 statistics
    use_beta: whether to use the beta and se or p value to calculate chi2
    
    Prints the intercept"""
    start = time.time()

    attenuation_df = pd.read_csv('disattenuation/standard_errors.txt', sep='\s+', comment='#')
    first_ldscores = get_first_weight(gwas, attenuation_df)
    gwas_variants = pd.read_csv(gwas + '/variants.txt', sep='\s+', comment='#', header=None)
    gwas_df = pd.read_csv(gwas_data + '/' + gwas_file, sep='\s+', comment='#')
    
    ldscores = load_ldscores()

    print("Done Loading", time.time() - start)
    start = time.time()

    merged_gwas = calc_chi2(gwas_df.merge(gwas_variants, left_on=gwas_snp, right_on=0).merge(ldscores, left_on=gwas_snp, right_on="SNP").merge(attenuation_df, left_on="SNP", right_on="SNP"), effect_size, standard_error, use_beta)
    
    print("Chi2 Obtained", time.time() - start, "SNPs", len(merged_gwas['SNP']))
    start = time.time()
    # print(merged_gwas['ldscore'])
    # Do the ldscores need to be fmaxed?
    M = gwas_variants[0].size
    N = 378
    second_weights = second_weight(merged_gwas['chi2'] *  gc, merged_gwas['ldscore'], M, N, merged_gwas['ldscore'])

    print("Second Weight Obtained", time.time() - start)
    start = time.time()

    weights = 1 / (np.multiply(np.array(np.fmax(first_ldscores + 1, 1.0)), second_weights)) 
    #weights = np.array(np.fmax(first_ldscores, 1)) * 2 * second_weights
    # print(np.isnan(np.fmax(first_ldscores, 1)).any(), np.isnan(second_weights).any(), np.isnan(weights).any(), np.isnan(merged_gwas['chi2']).any(), np.isnan(merged_gwas['ldscore']).any())

    disattenuation = 1 / np.sqrt(np.multiply(np.square(merged_gwas['atten']), second_weights))

    weights = np.multiply(weights, disattenuation)

    print(weights)
    regression_output = regression_gwas(merged_gwas['chi2'] *  gc, merged_gwas['ldscore'], M, N, weights)

    print("Regression completed", time.time() - start)

    print(regression_output.params[0], regression_output.params[1] * N)










In [725]:
# Height
get_weights('height', "../gwas_files", 'GIANT_HEIGHT_LangoAllen2010_publicrelease_HapMapCeuFreq.txt', 'MarkerName', 'p', 'MarkerName', 1.478, False)

Done Loading 37.179243326187134
Chi2 Obtained 20.58875823020935 SNPs 1140214
param: const      1.430369
ldscore    0.004503
dtype: float64
Second Weight Obtained 0.09049558639526367
0          0.041845
1          0.027645
2          0.035661
3          0.050463
4          0.021751
             ...   
1140209    0.051854
1140210    0.011105
1140211    0.016889
1140212    0.031519
1140213    0.010782
Length: 1140214, dtype: float64


C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\928207817.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return np.square(np.multiply(N * min(max((M * results.params[1] / N ), 0), 1) / M, np.fmax(first_weight, 1.0)) + 1)
C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\2870451518.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(regression_output.params[0], regression_output.params[1] * N)


Regression completed 0.12083125114440918
1.3983410992436238 1.8520580349147884


In [726]:
# bmi
get_weights('bmi', "../gwas_files", 'GIANT_BMI_Speliotes2010_publicrelease_HapMapCeuFreq.txt', 'MarkerName', 'p', 'MarkerName', 1.090, False)

Done Loading 61.87524390220642
Chi2 Obtained 22.804320335388184 SNPs 1140975
param: const      1.017011
ldscore    0.001596
dtype: float64
Second Weight Obtained 0.09157586097717285
0          0.041807
1          0.027628
2          0.035660
3          0.050588
4          0.021806
             ...   
1140970    0.051895
1140971    0.011106
1140972    0.016893
1140973    0.031487
1140974    0.010771
Length: 1140975, dtype: float64


C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\928207817.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return np.square(np.multiply(N * min(max((M * results.params[1] / N ), 0), 1) / M, np.fmax(first_weight, 1.0)) + 1)
C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\2870451518.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(regression_output.params[0], regression_output.params[1] * N)


Regression completed 0.12433362007141113
0.9965900377349166 0.6950661452212179


In [727]:
# waist
get_weights('waist', "../gwas_files", 'GIANT_WHRadjBMI_Heid2010_publicrelease_HapMapCeuFreq.txt', 'MarkerName', 'p', 'MarkerName', 1.330, False)

Done Loading 61.36721897125244
Chi2 Obtained 22.743091821670532 SNPs 1126058
param: const      1.330762
ldscore    0.000510
dtype: float64
Second Weight Obtained 0.0887296199798584
0          0.041801
1          0.027749
2          0.038457
3          0.050876
4          0.023783
             ...   
1126053    0.039964
1126054    0.016948
1126055    0.015111
1126056    0.031658
1126057    0.013064
Length: 1126058, dtype: float64
Regression completed 0.1157999038696289
1.3239821798346547 0.21938783947550006


C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\928207817.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return np.square(np.multiply(N * min(max((M * results.params[1] / N ), 0), 1) / M, np.fmax(first_weight, 1.0)) + 1)
C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\2870451518.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(regression_output.params[0], regression_output.params[1] * N)


In [728]:
# education
get_weights('education', "../gwas_files", 'SSGAC_EduYears_Rietveld2013_publicrelease.txt', 'MarkerName', 'Pvalue', 'MarkerName', 1.220, False)

Done Loading 49.66522979736328
Chi2 Obtained 11.720123291015625 SNPs 1088091
param: const      1.372808
ldscore    0.001092
dtype: float64
Second Weight Obtained 0.05253195762634277
0          0.036013
1          0.061763
2          0.037009
3          0.024561
4          0.018799
             ...   
1088086    0.032514
1088087    0.033159
1088088    0.015475
1088089    0.032842
1088090    0.041009
Length: 1088091, dtype: float64
Regression completed 0.06591534614562988
1.3156363360746206 0.4716622269183539


C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\928207817.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return np.square(np.multiply(N * min(max((M * results.params[1] / N ), 0), 1) / M, np.fmax(first_weight, 1.0)) + 1)
C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\2870451518.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(regression_output.params[0], regression_output.params[1] * N)


In [729]:
# college
get_weights('college', "../gwas_files", 'SSGAC_College_Rietveld2013_publicrelease.txt', 'MarkerName', 'Pvalue', 'MarkerName', 1.207, False)

Done Loading 37.295104026794434
Chi2 Obtained 11.79168152809143 SNPs 1091686
param: const      1.343896
ldscore    0.001118
dtype: float64
Second Weight Obtained 0.05286836624145508
0          0.033564
1          0.060763
2          0.034059
3          0.022572
4          0.017526
             ...   
1091681    0.032564
1091682    0.033194
1091683    0.015456
1091684    0.032746
1091685    0.040882
Length: 1091686, dtype: float64
Regression completed 0.0703272819519043
1.2983511214306966 0.44576783783199453


C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\928207817.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return np.square(np.multiply(N * min(max((M * results.params[1] / N ), 0), 1) / M, np.fmax(first_weight, 1.0)) + 1)
C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\2870451518.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(regression_output.params[0], regression_output.params[1] * N)


In [730]:
# fasting
get_weights('fasting', "../gwas_files", 'MAGIC_ln_FastingInsulin.txt', 'snp', 'pvalue', 'snp', 1.041, False)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
# artery
get_weights('artery', "../gwas_files", 'CARDIoGRAM_GWAS_RESULTS.txt', 'SNP', 'pvalue', 'SNP', 1.125, False)

Done Loading 40.671815633773804
Chi2 Obtained 12.6837317943573 SNPs 1126872
param: const      1.197695
ldscore    0.000599
dtype: float64
Second Weight Obtained 0.056548118591308594
0          0.065954
1          0.065638
2          0.044278
3          0.029131
4          0.055310
             ...   
1126867    0.032265
1126868    0.033696
1126869    0.042572
1126870    0.031324
1126871    0.039161
Length: 1126872, dtype: float64
Regression completed 0.0716552734375
1.1680078836643801 2.2266452330875612


C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\928207817.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return np.square(np.multiply(N * min(max((M * results.params[1] / N ), 0), 1) / M, np.fmax(first_weight, 1.0)) + 1)
C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\3283508164.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(regression_output.params[0], regression_output.params[1] * M / N)


In [ ]:
# depression
get_weights('depression', "../gwas_files", 'pgc.mdd.full.2012-04.txt', 'snpid', 'pval', 'snpid', 1, False)

Done Loading 36.74102711677551
Chi2 Obtained 11.378296852111816 SNPs 1161115
param: const      1.016590
ldscore    0.000357
dtype: float64
Second Weight Obtained 0.05639934539794922
0          0.080440
1          0.028581
2          0.047852
3          0.037066
4          0.048094
             ...   
1161110    0.038790
1161111    0.083991
1161112    0.051395
1161113    0.053418
1161114    0.053707
Length: 1161115, dtype: float64
Regression completed 0.07138252258300781
0.9994790003665663 1.0759722415327613


C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\928207817.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return np.square(np.multiply(N * min(max((M * results.params[1] / N ), 0), 1) / M, np.fmax(first_weight, 1.0)) + 1)
C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\3283508164.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(regression_output.params[0], regression_output.params[1] * M / N)


In [ ]:
# cross
get_weights('cross', "../gwas_files", 'pgc.cross.full.2013-03.txt', 'snpid', 'pval', 'snpid', 1, False)

Done Loading 36.99411416053772
Chi2 Obtained 11.744894027709961 SNPs 1165995
param: const      1.080475
ldscore    0.001011
dtype: float64
Second Weight Obtained 0.056765079498291016
0          0.080039
1          0.089529
2          0.028432
3          0.047128
4          0.035724
             ...   
1165990    0.038897
1165991    0.084164
1165992    0.051396
1165993    0.053537
1165994    0.053758
Length: 1165995, dtype: float64
Regression completed 0.07088255882263184
1.0351757903257028 3.7496047861046558


C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\928207817.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return np.square(np.multiply(N * min(max((M * results.params[1] / N ), 0), 1) / M, np.fmax(first_weight, 1.0)) + 1)
C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\3283508164.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(regression_output.params[0], regression_output.params[1] * M / N)


In [ ]:
# bipolar
get_weights('bipolar', "../gwas_files", 'pgc.bip.full.2012-04.txt', 'snpid', 'pval', 'snpid', 1, False)

Done Loading 38.52571439743042
Chi2 Obtained 11.937570095062256 SNPs 1134890
param: const      1.073080
ldscore    0.000695
dtype: float64
Second Weight Obtained 0.05725884437561035
0          0.066219
1          0.091289
2          0.065058
3          0.043758
4          0.037499
             ...   
1134885    0.015067
1134886    0.040876
1134887    0.031194
1134888    0.038980
1134889    0.056707
Length: 1134890, dtype: float64
Regression completed 0.07067704200744629
1.0445012638889848 2.5108827540926892


C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\928207817.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return np.square(np.multiply(N * min(max((M * results.params[1] / N ), 0), 1) / M, np.fmax(first_weight, 1.0)) + 1)
C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\3283508164.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(regression_output.params[0], regression_output.params[1] * M / N)


In [ ]:
# adhd
get_weights('adhd', "../gwas_files", 'pgc.adhd.full.2012-10.txt', 'snpid', 'pval', 'snpid', 1, False)

Done Loading 36.37403631210327
Chi2 Obtained 11.451959133148193 SNPs 1146435
param: const      1.019998
ldscore    0.000070
dtype: float64
Second Weight Obtained 0.05543923377990723
0          0.081420
1          0.094419
2          0.029477
3          0.049900
4          0.037466
             ...   
1146430    0.033765
1146431    0.031411
1146432    0.039236
1146433    0.034110
1146434    0.042721
Length: 1146435, dtype: float64
Regression completed 0.06944727897644043
1.000252913068115 0.3873870837301975


C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\928207817.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return np.square(np.multiply(N * min(max((M * results.params[1] / N ), 0), 1) / M, np.fmax(first_weight, 1.0)) + 1)
C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\3283508164.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(regression_output.params[0], regression_output.params[1] * M / N)


In [ ]:
# scz
get_weights('scz', "../gwas_files", 'pgc.scz.full.2012-04.txt', 'snpid', 'pval', 'snpid', 1, False)

Done Loading 36.963457345962524
Chi2 Obtained 11.953407764434814 SNPs 1166286
param: const      1.076245
ldscore    0.001497
dtype: float64
Second Weight Obtained 0.0581212043762207
0          0.080162
1          0.089755
2          0.028954
3          0.047274
4          0.035964
             ...   
1166281    0.038897
1166282    0.084165
1166283    0.051396
1166284    0.053537
1166285    0.053758
Length: 1166286, dtype: float64
Regression completed 0.07321381568908691
1.046380864303043 4.920615175461235


C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\928207817.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return np.square(np.multiply(N * min(max((M * results.params[1] / N ), 0), 1) / M, np.fmax(first_weight, 1.0)) + 1)
C:\Users\Lingxiao Guan\AppData\Local\Temp\ipykernel_33764\3283508164.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(regression_output.params[0], regression_output.params[1] * M / N)
